# Table of contents

## Importing libraries

In [19]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler

## Loading data

In [20]:
df_train = pd.read_csv('./csv/conjunto_de_treinamento.csv')
df_test = pd.read_csv('./csv/conjunto_de_teste.csv')

In [21]:
df_train.shape, df_test.shape

((20000, 42), (5000, 41))

## First 5 rows of training dataset

In [22]:
df_train.head()

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
0,1,1,10,presencial,1,M,85,2,0,0,...,N,,0,9.0,1.0,0.0,0.0,600.0,600.0,0
1,2,1,25,internet,1,F,38,1,0,0,...,N,,0,2.0,5.0,NaN,NaN,492.0,492.0,0
2,3,1,20,internet,1,F,37,2,0,0,...,N,,0,NaN,NaN,NaN,NaN,450.0,450.0,1
3,4,1,20,internet,1,M,37,1,1,0,...,Y,54,0,9.0,2.0,NaN,NaN,932.0,932.0,1
4,5,7,1,internet,1,F,51,1,3,0,...,N,,0,9.0,5.0,NaN,NaN,440.0,440.0,1


## Summary statistics for numerical columns

In [23]:
num_cols = df_train.select_dtypes(include=np.number).columns
df_train[num_cols].describe()

,id_solicitante,produto_solicitado,dia_vencimento,tipo_endereco,idade,estado_civil,qtde_dependentes,grau_instrucao,nacionalidade,tipo_residencia,...,valor_patrimonio_pessoal,possui_carro,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.0,20000.000000,19464.000000,...,2.000000e+04,20000.000000,20000.000000,16903.000000,17022.000000,8486.000000,7140.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,1.284350,13.147250,1.006350,42.352500,2.120850,0.666400,0.0,0.961600,1.261303,...,2.095614e+03,0.339050,0.008900,8.045081,2.533310,3.708107,0.288095,581.295250,581.295250,0.500000
std,5773.647028,1.008239,6.748507,0.079435,14.930177,1.332004,1.236725,0.0,0.202305,0.883580,...,4.403344e+04,0.473399,0.388881,3.210790,1.532765,5.181241,0.944339,227.369798,227.369798,0.500013
min,1.000000,1.000000,1.000000,1.000000,7.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,105.000000,105.000000,0.000000
25%,5000.750000,1.000000,10.000000,1.000000,31.000000,1.000000,0.000000,0.0,1.000000,1.000000,...,0.000000e+00,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,444.000000,444.000000,0.000000
50%,10000.500000,1.000000,10.000000,1.000000,40.000000,2.000000,0.000000,0.0,1.000000,1.000000,...,0.000000e+00,0.000000,0.000000,9.000000,2.000000,0.000000,0.000000,596.000000,596.000000,0.500000
75%,15000.250000,1.000000,20.000000,1.000000,52.000000,2.000000,1.000000,0.0,1.000000,1.000000,...,0.000000e+00,1.000000,0.000000,9.000000,4.000000,11.000000,0.000000,728.000000,728.000000,1.000000
max,20000.000000,7.000000,25.000000,2.000000,106.000000,7.000000,53.000000,0.0,2.000000,5.000000,...,6.000000e+06,1.000000,32.000000,17.000000,5.000000,17.000000,5.000000,999.000000,999.000000,1.000000


## Identify categorical variables and print unique values and their counts

In [24]:
cat_cols = df_train.select_dtypes(include='object').columns
for col in cat_cols:
    display(f'{col} unique values and counts:', df_train[col].value_counts())

'forma_envio_solicitacao unique values and counts:'

internet      11264
presencial     7855
correio         881
Name: forma_envio_solicitacao, dtype: int64

'sexo unique values and counts:'

F    12246
M     7722
N       25
         7
Name: sexo, dtype: int64

'estado_onde_nasceu unique values and counts:'

...

      13573
SP     1010
RS      819
CE      588
BA      569
MG      500
PE      369
PA      316
PR      236
RJ      229
MT      224
GO      218
RN      212
AL      151
MS      150
PB      132
DF      115
SC      114
ES       86
AP       77
MA       73
RO       61
AM       54
PI       39
AC       36
SE       25
TO       18
RR        6
Name: estado_onde_trabalha, dtype: int64

'possui_telefone_trabalho unique values and counts:'

N    14519
Y     5481
Name: possui_telefone_trabalho, dtype: int64

'codigo_area_telefone_trabalho unique values and counts:'

       14525
5        631
54       442
107      407
97       264
       ...  
115        1
96         1
53         1
122        1
74         1
Name: codigo_area_telefone_trabalho, Length: 77, dtype: int64

# Preprocess the data

In [26]:
scaler = MinMaxScaler()
df_train[num_cols] = scaler.fit_transform(df_train[num_cols])

df_train[num_cols] = scaler.transform(df_train[num_cols])
df_test[num_cols] = scaler.transform(df_test[num_cols])

label_bin = LabelBinarizer()
df_train[cat_cols] = label_bin.fit_transform(df_train[cat_cols])

df_test[cat_cols] = label_bin.transform(df_test[cat_cols])

KeyError: "['inadimplente'] not in index"

## Define high cardinality columns to drop

In [ ]:
high_card_cols = ['estado_onde_nasceu', 'estado_onde_reside', 'codigo_area_telefone_residencial',
                  'possui_telefone_celular', 'estado_onde_trabalha', 'codigo_area_telefone_trabalho',
                  'qtde_contas_bancarias_especiais', 'grau_instrucao', 'meses_no_trabalho']

## Drop high cardinality columns

In [ ]:
dados.drop(high_card_cols, axis=1, inplace=True)
resposta.drop(high_card_cols, axis=1, inplace=True)

## Apply one-hot encoding to categorical columns with 3 or more categories

In [ ]:
cat_cols = ['sexo', 'forma_envio_solicitacao']
dados = pd.get_dummies(dados, columns=cat_cols)
resposta = pd.get_dummies(resposta, columns=cat_cols)

## Apply binary encoding to binary columns

In [ ]:
bin_cols = ['possui_telefone_residencial', 'vinculo_formal_com_empresa', 'possui_telefone_trabalho']
binarizador = LabelBinarizer()
for col in bin_cols:
    dados[col] = binarizador.fit_transform(dados[col])
    resposta[col] = binarizador.fit_transform(resposta[col])

## Fill missing values in selected columns with 0 or 'N'

In [ ]:
missing_cols = ['profissao_companheiro', 'grau_instrucao_companheiro']
dados[missing_cols] = dados[missing_cols].fillna(0)
resposta[missing_cols] = resposta[missing_cols].fillna(0)
dados['sexo'] = dados['sexo'].fillna('N')
resposta['sexo'] = resposta['sexo'].fillna('N')

## Display preprocessed data

In [ ]:
display(dados.head(5).T)

## Check class distribution

In [ ]:
print("Class distribution:")
display(dados['inadimplente'].value_counts())

## Check mean values of attributes in each class

In [ ]:
print("\nMean values of attributes in each class:")
display(dados.groupby(['inadimplente']).mean().T)